In [ ]:
import os
from ultralytics import YOLO


In [ ]:
# Configuración
image_folder = "dataset/samples"  # Carpeta con las imágenes
output_file = "alt_texts.txt"  # Archivo donde se guardarán los textos alternativos
model_path = "model/yolov8x.pt"  # Ruta al modelo YOLO (puedes usar un modelo preentrenado como yolov8x.pt)

# Cargar el modelo
model = YOLO(model_path)


In [ ]:

# Crear o vaciar el archivo de salida
with open(output_file, "w") as f:
    f.write("")  # Vaciar el archivo


In [ ]:

# Procesar las imágenes
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    
    # Asegurarse de que es una imagen
    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
        continue

    # Realizar detección con YOLO
    results = model(image_path)

    # Extraer información de detección
    alt_texts = []
    for result in results:
        for box in result.boxes.data:
            cls = int(box[5])  # Clase detectada
            confidence = float(box[4])  # Confianza de la predicción
            label = model.names[cls]  # Nombre de la clase
            alt_texts.append(f"{label} ({confidence:.2f})")
    
    # Crear el texto alternativo
    alt_text = f"{image_name}: {', '.join(alt_texts)}"
    
    # Guardar en el archivo de salida
    with open(output_file, "a") as f:
        f.write(alt_text + "\n")
    
    print(f"Procesada: {image_name} -> {alt_text}")
